### +++ static +++

In [27]:
import opensim as osim
import os
from simFunctions import runProgram, scaleOptimalForceSubjectSpecific
# from stan_utils import subject_specific_isometric_force
import pandas as pd
os.getcwd()

'c:\\Git\\1_current_projects\\powerlifing_model\\models\\athlete_03\\001bis\\P01\\model_update'

In [28]:
# ensure correct individual name
cwd = os.getcwd()
root = os.path.abspath(os.path.join(cwd, "..\\..\\..\\..\\athlete_03"))
tmp = cwd.split('\\')[len(root.split('\\')):]
ind = 'P' + tmp[0][1:]
ind

'P01bis'

# Udate Scaling XML

This file uses scaling scheme that is already given in scaling_setting.xml.

For walking, the model is scaled following distances between external markers and muscle forces are scaled by a factor calculated with mass and height of the subject relative the mass and the height of the generic model.

In [29]:
# IMPORTANT : THESE SHOULD BE PERSON-SPECIFIC
mass_text = '100'
height_text = '1.83'
age_text = '30'

# path to experimental .trc file
simulations_path = os.path.abspath(os.path.join(root, '..\\..\\simulations'))
experimental_markers = os.path.abspath(os.path.join(simulations_path,'.\\Athlete_03\\22_07_06\\static_01\\marker_experimental.trc'))
static_df = pd.read_csv(experimental_markers, delimiter='\t', skiprows=3, header=[0,1], index_col=0)


In [30]:
# paths to and names of files
path_to_generic_model = os.path.join(root, '..\\Athlete_03_lowerBody_final.osim')
path_to_generic_marker_set =  os.path.join(root, '..\\Athlete_03_markers_lowerbody_katya.xml')

# <time_range>'
time_range = f' 0 {list(static_df.loc[static_df.shape[0], "Time"])[0] }'

# <output_scale_file>
path_to_model = '3a_osim_markers'

output_scale_file = os.path.join(path_to_model, 'output_scale_file.txt')

output_model_file = os.path.join(path_to_model, f'scaled_model_{ind}.osim')  # f'scaled_model_{ind}_joints.osim'

output_scaling_settings = os.path.join(path_to_model, f'scaling_setting_{ind}.xml')

if not os.path.exists(path_to_generic_model):
    print(f"Generic model not found at {path_to_generic_model}. Please check the path.")

if not os.path.exists(path_to_generic_marker_set):
    print(f"Generic marker set not found at {path_to_generic_marker_set}. Please check the path.")
    

In [31]:
output_scaling_settings


'3a_osim_markers\\scaling_setting_P01bis.xml'

In [32]:
import xml.etree.ElementTree as ET

# parse the .xml with markers in body frames
scaling_tree=ET.parse("../templates/scaling_setting.xml")
scaling_root = scaling_tree.getroot()

for generic_model in scaling_root.iter('model_file'):
    generic_model.text = path_to_generic_model

for generic_marker_set in scaling_root.iter('marker_set_file'):
    generic_marker_set.text = path_to_generic_marker_set

for exp_markers in scaling_root.iter('marker_file'):
    exp_markers.text = experimental_markers

for time in scaling_root.iter('time_range'):
    time.text = time_range

for output in scaling_root.iter('output_model_file'):
    output.text = output_model_file

for scale in scaling_root.iter('output_scale_file'):
    scale.text = output_scale_file

for generic_marker_set in scaling_root.iter('marker_set_file'):
    generic_marker_set.text = path_to_generic_marker_set

for exp_markers in scaling_root.iter('marker_file'):
    exp_markers.text = experimental_markers

for time in scaling_root.iter('time_range'):
    time.text = time_range

for output in scaling_root.iter('output_model_file'):
    output.text = output_model_file

for scale in scaling_root.iter('output_scale_file'):
    scale.text = output_scale_file

for marker_placer in scaling_root.iter('MarkerPlacer'):
    apply = marker_placer.find('apply')
    apply.text = 'true'

In [33]:
for mass in scaling_root.iter('mass'):
	mass.text = mass_text

for height in scaling_root.iter('height'):
	height.text = height_text

for age in scaling_root.iter('age'):
	age.text = age_text

In [34]:
scaling_tree.write(output_scaling_settings)

## 2.1 Scale model to markers

In [35]:
cmdprog = 'opensim-cmd'
cmdtool = 'run-tool'
cmdfile = output_scaling_settings
cmdfull = [cmdprog, cmdtool, cmdfile]
rc = runProgram(cmdfull)

c:\Users\Bas\miniconda3\envs\py311\Lib\subprocess.py:1016: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdout = io.open(c2pread, 'rb', bufsize)


[info] Preparing to run ScaleTool.

[info] Processing subject Rajagopal-generic-scaled...

[info] Step 1: Loading generic model

[info] Loaded model Athlete_03 from file c:\Git\1_current_projects\powerlifing_model\models\Athlete_03_lowerBody_final.osim

[info] Loading marker set from 'c:\Git\1_current_projects\powerlifing_model\models\athlete_03\..\Athlete_03_markers_lowerbody_katya.xml'.

[info] Updated markers in model Athlete_03

[info] Step 2: Scaling generic model

[info] Loaded marker file c:\Git\1_current_projects\powerlifing_model\simulations\Athlete_03\22_07_06\static_01\marker_experimental.trc (73 markers, 693 frames)

[info] Measurement 'pelvis width'

[info] 	pair 0 (LASI, RASI): model = 0.227655, experimental = 0.227666

[info] 	pair 1 (LPSI, RPSI): model = 0.0775916, experimental = 0.0891936

[info] 	overall scale factor = 1.07479

[info] Measurement 'pelvis height'

[warning] Marker LMKNE in pelvis height measurement not found in Rajagopal-generic-scaled.

[warning] Mark

## Scale model muscle forces to height and weight

In [36]:
generic_model = osim.Model(path_to_generic_model)
model=osim.Model(output_model_file)
scaled = scaleOptimalForceSubjectSpecific(generic_model, model, 1.77, float(height_text))


# Set "ignore_tendon_compliance" and "ignore_activation_dynamics" to "true"

In [37]:
muscle_set = scaled.getMuscles()
for i in range(muscle_set.getSize()):
    current = muscle_set.get(i)
    current.set_ignore_activation_dynamics(True)
    current.set_ignore_tendon_compliance(True)

## Increase Pelvis Tilt

In [41]:
# pelvis_coordinates = model.getCoordinateSet().get('pelvis_tilt')
# pelvis_coordinates.setRangeMax(4)
# pelvis_coordinates.setRangeMin(-4)

## Save the model

In [42]:
scaled.printToXML(output_model_file)

True

## Scale muscle fibres

In [43]:
# from fibre_scale_script import *
# # define models
# generic_model = '../templates/RajagopalModified_generic_copy.osim'
# model_to_update = output_model_file
# updated_model = output_model_file

# # run scripts
# osimModel_opt, SimInfo = optimMuscleParams(generic_model, model_to_update, 2, '3a_osim_markers/logging')

# # printing the optimized model
# osimModel_opt.printToXML(updated_model)

# Add joint center markers and save another model

In [44]:
model = osim.Model(output_model_file)
for body in model.getBodySet():
    if body.getName() == 'torso':
        torso_body = body

    if body.getName() == 'pelvis':
        pelvis_body = body
    if body.getName() == 'femur_r':
        femur_r_body = body
    if body.getName() == 'femur_l':
        femur_l_body = body

    if body.getName() == 'patella_r':
        patella_r_body = body
    if body.getName() == 'patella_l':
        patella_l_body = body

    if body.getName() == 'tibia_r':
        tibia_r_body = body
    if body.getName() == 'tibia_l':
        tibia_l_body = body

    if body.getName() == 'talus_r':
        talus_r_body = body
    if body.getName() == 'talus_l':
        talus_l_body = body

In [45]:
# torso_m = osim.Marker("torso_origin_in_pelvis", torso_body, osim.Vec3(0,0,0))
femur_l_m = osim.Marker("femur_l_center_in_pelvis", femur_l_body, osim.Vec3(0,0,0))
femur_r_m = osim.Marker("femur_r_center_in_pelvis", femur_r_body, osim.Vec3(0,0,0))
patella_l_m = osim.Marker("patella_l_in_femur_l", patella_l_body, osim.Vec3(0,0,0))
patella_r_m = osim.Marker("patella_r_in_femur_r", patella_r_body, osim.Vec3(0,0,0))
tibia_l_m = osim.Marker("knee_l_center_in_femur_l", tibia_l_body, osim.Vec3(0,0,0))
tibia_r_m = osim.Marker("knee_r_center_in_femur_r", tibia_r_body, osim.Vec3(0,0,0))
talus_l_m = osim.Marker("talus_l_center_in_tibia", talus_l_body, osim.Vec3(0,0,0))
talus_r_m = osim.Marker("talus_r_center_in_tibia", talus_r_body, osim.Vec3(0,0,0))

# model.addMarker(torso_m)
model.addMarker(femur_l_m)
model.addMarker(femur_r_m)
model.addMarker(patella_l_m)
model.addMarker(patella_r_m)
model.addMarker(tibia_l_m)
model.addMarker(tibia_r_m)
model.addMarker(talus_l_m)
model.addMarker(talus_r_m)

model.finalizeConnections()

In [46]:
state = model.initSystem()

# torso_m.changeFramePreserveLocation(state, pelvis_body)
femur_l_m.changeFramePreserveLocation(state, pelvis_body)
femur_r_m.changeFramePreserveLocation(state, pelvis_body)
patella_l_m.changeFramePreserveLocation(state, femur_l_body)
patella_r_m.changeFramePreserveLocation(state, femur_r_body)
tibia_l_m.changeFramePreserveLocation(state, femur_l_body)
tibia_r_m.changeFramePreserveLocation(state, femur_r_body)
talus_l_m.changeFramePreserveLocation(state, tibia_l_body)
talus_r_m.changeFramePreserveLocation(state, tibia_r_body)

In [47]:
model.finalizeConnections()
model.printToXML(os.path.join(path_to_model, f'scaled_model_{ind}_joints.osim'))

True

## Output model markers

In [49]:
model=osim.Model(os.path.join(path_to_model, f'scaled_model_{ind}_joints.osim'))
set = model.get_MarkerSet()
lms = ['RASI', 'LASI', 'RPSI', 'LPSI', 'torso_origin_in_pelvis', 'femur_l_center_in_pelvis', 'femur_r_center_in_pelvis',
       'knee_l_center_in_femur_l', 'knee_r_center_in_femur_r', 
       'talus_l_center_in_tibia', 'talus_r_center_in_tibia']

list_of_rows = []
index = 0
for marker in set:
    name = marker.getName()
    if name in lms:
        body = (marker.getParentFrameName()).split('/')[-1]
        location = marker.get_location().to_numpy()
        x,y,z = location
        new_row = pd.DataFrame([[name, body, x, y, z]], columns =['name', 'body', 'x', 'y', 'z'] )
        list_of_rows.append(new_row)
        
df = pd.concat(list_of_rows)
df = df.set_index('name')
df.to_csv(f'{root}/{ind}_generic_landmarks.csv')